# Finding Data Block Nirvana (a journey through the fastai data block API) - Part 1

This notebook illustrates how to create a custom `ItemList` for use in the fastai data block API.  It is heavily annotated to further aid in also understanding how all the different bits in the API interact as well as what is happening at each step and why.

Please consult the [fastai docs](https://docs.fast.ai/) for installing required packages and setting up your environment to run the code below.

The accompanying Medium article highlighing the data block API mechanics based on my work here can be found [here](https://medium.com/@wgilliam/finding-data-block-nirvana-a-journey-through-the-fastai-data-block-api-c38210537fe4).

**Update - 10/31/2019**

**Part 2** of the accompanying Medium article describing how to use the custom ItemList created in Part 1 can be found [here](https://medium.com/@wgilliam/finding-data-block-nirvana-a-journey-through-the-fastai-data-block-api-part-2-9b23ea5d83ee).

I've also made some fixes to the `mixed_tabular_pad_collate` function (see below).  Essentially, the tensors weren't arranged properly for training.  I've kept the wrong code in there as well so you can play with it ... definitely helpful to understand and see the differences.

## Yelp Dataset

This example utilize a subset of the Yelp review dataset I've made available as part of the code repo for the purposes of illustrating how my `MixedTabularList` would work with a pandas DataFrame containing categorical, continuous, and numercalized text data.  The full dataset and documentation can be found following the links below.

Available from https://www.yelp.com/dataset/download  
Documentation here:  https://www.yelp.com/dataset/documentation/main  
More information here:  https://www.yelp.com/dataset

Unzip the `joined_sample.zip` .csv file into a `data/yelp_dataset` folder relative to this notebook and you should be good to go.

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pdb

from fastai.tabular import *
from fastai.text import *
from fastai.text.data import _join_texts

print(f'fastai version: {__version__}')  #=> I test this against 1.0.59

fastai version: 1.0.59


In [3]:
torch.cuda.set_device(1)
print(f'using GPU: {torch.cuda.current_device()}')

using GPU: 1


## Configuration and utility methods

In [4]:
PATH=Path('data/yelp_dataset/')
PATH.ls()

[PosixPath('data/yelp_dataset/data_lm_export.pkl'),
 PosixPath('data/yelp_dataset/joined_sample.csv'),
 PosixPath('data/yelp_dataset/models'),
 PosixPath('data/yelp_dataset/tmp')]

## Define ItemBase subclass

`ItemBase` defines the inputs for your custom dataset, the X and optionally y values you are going to feed into the `forward` function of your pytorch model.  Here we define what an an input item looks like (we'll let fastai infer the `ItemBase` type to use based on our target values).

If your custom `ItemBase` needs to have some kind of data augmentation applied to it, you should overload the `apply_tfms` method as needed.  This method will be called you apply a `transform` block via the Data Block API.

In [5]:
class MixedTabularLine(TabularLine):
    "Item's that include both tabular data(`conts` and `cats`) and textual data (numericalized `ids`)"
    
    def __init__(self, cats, conts, cat_classes, col_names, txt_ids, txt_cols, txt_string):
        # tabular
        super().__init__(cats, conts, cat_classes, col_names)

        # add the text bits
        self.text_ids = txt_ids
        self.text_cols = txt_cols
        self.text = txt_string
        
        # append numericalted text data to your input (represents your X values that are fed into your model)
        # self.data = [tensor(cats), tensor(conts), tensor(txt_ids)]
        self.data += [ np.array(txt_ids, dtype=np.int64) ]
        self.obj = self.data
        
    def __str__(self):
        res = super().__str__() + f'Text: {self.text}'
        return res

## Define custom Processor, DataBunch, and utility methods

Our custom `ItemList` is going to require a custom `PreProcessor` and a custom `DataBunch`, so we define them here

In [6]:
class MixedTabularProcessor(TabularProcessor):
    
    def __init__(self, ds:ItemList=None, procs=None, 
                 tokenizer:Tokenizer=None, chunksize:int=10000,
                 vocab:Vocab=None, max_vocab:int=60000, min_freq:int=2):
        #pdb.set_trace()
        super().__init__(ds, procs)
    
        self.tokenizer, self.chunksize = ifnone(tokenizer, Tokenizer()), chunksize
        
        vocab = ifnone(vocab, ds.vocab if ds is not None else None)
        self.vocab, self.max_vocab, self.min_freq = vocab, max_vocab, min_freq
        
    # process a single item in a dataset
    # NOTE: THIS IS METHOD HAS NOT BEEN TESTED AT THIS POINT (WILL COVER IN A FUTURE ARTICLE)
    def process_one(self, item):
        # process tabular data (copied form tabular.data)
        df = pd.DataFrame([item, item])
        for proc in self.procs: proc(df, test=True)
            
        if len(self.cat_names) != 0:
            codes = np.stack([c.cat.codes.values for n,c in df[self.cat_names].items()], 1).astype(np.int64) + 1
        else: 
            codes = [[]]
            
        if len(self.cont_names) != 0:
            conts = np.stack([c.astype('float32').values for n,c in df[self.cont_names].items()], 1)
        else: 
            conts = [[]]
            
        classes = None
        col_names = list(df[self.cat_names].columns.values) + list(df[self.cont_names].columns.values)
        
        # process textual data
        if len(self.text_cols) != 0:
            txt = _join_texts(df[self.text_cols].values, (len(self.text_cols) > 1))
            txt_toks = self.tokenizer._process_all_1(txt)[0]
            text_ids = np.array(self.vocab.numericalize(txt_toks), dtype=np.int64)
        else:
            txt_toks, text_ids = None, [[]]
            
        # return ItemBase
        return MixedTabularLine(codes[0], conts[0], classes, col_names, text_ids, self.txt_cols, txt_toks)
    
    # processes the entire dataset
    def process(self, ds):
        #pdb.set_trace()
        # process tabular data and then set "preprocessed=False" since we still have text data possibly
        super().process(ds)
        ds.preprocessed = False
        
        # process text data from column(s) containing text
        if len(ds.text_cols) != 0:
            texts = _join_texts(ds.inner_df[ds.text_cols].values, (len(ds.text_cols) > 1))

            # tokenize (set = .text)
            tokens = []
            for i in progress_bar(range(0, len(ds), self.chunksize), leave=False):
                tokens += self.tokenizer.process_all(texts[i:i+self.chunksize])
            ds.text = tokens

            # set/build vocab
            if self.vocab is None: self.vocab = Vocab.create(ds.text, self.max_vocab, self.min_freq)
            ds.vocab = self.vocab
            ds.text_ids = [ np.array(self.vocab.numericalize(toks), dtype=np.int64) for toks in ds.text ]
        else:
            ds.text, ds.vocab, ds.text_ids = None, None, []
            
        ds.preprocessed = True
        

In [7]:
# similar to the "fasta.text.data.pad_collate" except that it is designed to work with MixedTabularLine items,
# where the final thing in an item is the numericalized text ids.
# we need a collate function to ensure a square matrix with the text ids, which will be of variable length.
def mixed_tabular_pad_collate(samples:BatchSamples, 
                              pad_idx:int=1, pad_first:bool=True) -> Tuple[LongTensor, LongTensor]:
    "Function that collect samples and adds padding."

    samples = to_data(samples)
    max_len = max([len(s[0][-1]) for s in samples])
    res = torch.zeros(len(samples), max_len).long() + pad_idx
   
    for i,s in enumerate(samples):
        if pad_first: 
            res[i,-len(s[0][-1]):] = LongTensor(s[0][-1])
        else:         
            res[i,:len(s[0][-1]):] = LongTensor(s[0][-1])
            
        # replace the text_ids array (the last thing in the inputs) with the padded tensor matrix
        s[0][-1] = res[i]
             
    # 10/31/2019 (wtg) - keeping the old code so you can see the difference ...
    # for the inputs, return a list containing 3 elements: a list of cats, a list of conts, and a list of text_ids
    # return [x for x in zip(*[s[0] for s in samples])], tensor([s[1] for s in samples])

    cats = torch.cat([ s[0][0].unsqueeze(0) for s in samples ], 0)
    conts = torch.cat([ s[0][1].unsqueeze(0) for s in samples ], 0)
    texts = torch.cat([ s[0][2].unsqueeze(0) for s in samples ], 0)
    return [cats, conts, texts], tensor([ s[1] for s in samples ])

In [8]:
# each "ds" is of type LabelList(Dataset)
class MixedTabularDataBunch(DataBunch):
    @classmethod
    def create(cls, train_ds, valid_ds, test_ds=None, path:PathOrStr='.', bs=64, 
               pad_idx=1, pad_first=True, no_check:bool=False, **kwargs) -> DataBunch:
        
        # only thing we're doing here is setting the collate_fn = to our new "pad_collate" method above
        collate_fn = partial(mixed_tabular_pad_collate, pad_idx=pad_idx, pad_first=pad_first)
        
        kwargs['collate_fn'] = collate_fn
        kwargs['num_workers'] = 1
        return super().create(train_ds, valid_ds, test_ds, path=path, bs=bs, no_check=no_check, **kwargs)

## Define ItemList subclass

An `ItemList` consists of a set of `ItemBase` objects. Once created, you can use any of splitting or labeling methods prior to creating a `DataBunch` for training.

You'll likely want to set the following three class variables to something specific to your situation:

**`_bunch`**:  
The name of the class used to create a `DataBunch`.  `TabularList` uses the default `DataBunch` as is and so does not set this variable. We create a custom `DataBunch` here because we need to add padding to the column with the text ids in order to ensure a square matrix per batch before integrating the text bits with the tabular.

When you call `databunch()` via the Data Block API, `_bunch.create` will be called passing in the datasets (training, validation and optionally test) defined by your `ItemLists` and returning a set of `DataLoader`s in a `DataBunch` for training.

**`_processor`**:  
A class or list of classes of type `PreProcessor` that will be used to create the default processor for this `ItemList`.

The processors are **called at the end of the labelling** to apply some kind of function on your items. The **default processor of the inputs** can be overriden by passing a `processor` in the kwargs when creating the `ItemList`, the **default processor of the targets** can be overriden by passing a `processor` in the kwargs of the labelling function.

Processors are useful for pre-processing data, and **you also need to save any computed state required for future datasets when `data.export()` is called.**

**`_item_cls`**:   
The name of the class that will be used to create the "items" by default.

**`_label_cls`**:   
The name of the class that will be used to create the labels by default. (**If this variable is set to None, the label class will be guessed** between `CategoryList`, `MultiCategoryList` and `FloatList` depending on the type of the first item. Since we are creating a custom `ItemList` with a very distinct signature, we want to set it to that class)



In [9]:
class MixedTabularList(TabularList):
    "A custom `ItemList` that merges tabular data along with textual data"
    
    _item_cls = MixedTabularLine
    _processor = MixedTabularProcessor
    _bunch = MixedTabularDataBunch
    
    def __init__(self, items:Iterator, cat_names:OptStrList=None, cont_names:OptStrList=None, 
                 text_cols=None, vocab:Vocab=None, pad_idx:int=1, 
                 procs=None, **kwargs) -> 'MixedTabularList':
        #pdb.set_trace()
        super().__init__(items, cat_names, cont_names, procs, **kwargs)
        
        self.cols = [] if cat_names == None else cat_names.copy()
        if cont_names: self.cols += cont_names.copy()
        if txt_cols: self.cols += text_cols.copy()
        
        self.text_cols, self.vocab, self.pad_idx = text_cols, vocab, pad_idx
        
        # add any ItemList state into "copy_new" that needs to be copied each time "new()" is called; 
        # your ItemList acts as a prototype for training, validation, and/or test ItemList instances that
        # are created via ItemList.new()
        self.copy_new += ['text_cols', 'vocab', 'pad_idx']
        
        self.preprocessed = False
        
    # defines how to construct an ItemBase from the data in the ItemList.items array
    def get(self, i):
        if not self.preprocessed: 
            return self.inner_df.iloc[i][self.cols] if hasattr(self, 'inner_df') else self.items[i]
        
        codes = [] if self.codes is None else self.codes[i]
        conts = [] if self.conts is None else self.conts[i]
        text_ids = [] if self.text_ids is None else self.text_ids[i]
        text_string = None if self.text_ids is None else self.vocab.textify(self.text_ids[i])
        
        return self._item_cls(codes, conts, self.classes, self.col_names, text_ids, self.text_cols, text_string)
    
    # this is the method that is called in data.show_batch(), learn.predict() or learn.show_results() 
    # to transform a pytorch tensor back in an ItemBase. 
    # in a way, it does the opposite of calling ItemBase.data. It should take a tensor t and return 
    # the same king of thing as the get method.
    def reconstruct(self, t:Tensor):
        return self._item_cls(t[0], t[1], self.classes, self.col_names, 
                              t[2], self.text_cols, self.vocab.textify(t[2]))
    
    # tells fastai how to display a custom ItemBase when data.show_batch() is called
    def show_xys(self, xs, ys) -> None:
        "Show the `xs` (inputs) and `ys` (targets)."
        from IPython.display import display, HTML
        
        # show tabular
        display(HTML('TABULAR:<br>'))
        super().show_xys(xs, ys)
        
        # show text
        items = [['text_data', 'target']]
        for i, (x,y) in enumerate(zip(xs,ys)):
            res = []
            res += [' '.join([ f'{tok}({self.vocab.stoi[tok]})' 
                              for tok in x.text.split() if (not self.vocab.stoi[tok] == self.pad_idx) ])]
                
            res += [str(y)]
            items.append(res)
            
        col_widths = [90, 1]
        
        display(HTML('TEXT:<br>'))
        display(HTML(text2html_table(items)))
        
    # tells fastai how to display a custom ItemBase when learn.show_results() is called
    def show_xyzs(self, xs, ys, zs):
        "Show `xs` (inputs), `ys` (targets) and `zs` (predictions)."
        from IPython.display import display, HTML
        
        # show tabular
        super().show_xyzs(xs, ys, zs)
        
        # show text
        items = [['text_data','target', 'prediction']]
        for i, (x,y,z) in enumerate(zip(xs,ys,zs)):
            res = []
            res += [' '.join([ f'{tok}({self.vocab.stoi[tok]})'
                              for tok in x.text.split() if (not self.vocab.stoi[tok] == self.pad_idx) ])]
                
            res += [str(y),str(z)]
            items.append(res)
            
        col_widths = [90, 1, 1]
        display(HTML('<br>' + text2html_table(items)))
    
        
    @classmethod
    def from_df(cls, df:DataFrame, cat_names:OptStrList=None, cont_names:OptStrList=None, 
                text_cols=None, vocab=None, procs=None, **kwargs) -> 'ItemList':
        
        return cls(items=range(len(df)), cat_names=cat_names, cont_names=cont_names, 
                   text_cols=text_cols, vocab=vocab, procs=procs, inner_df=df, **kwargs)
    
    

## Fetch joined yelp reviews (includes busines and user info)

In [10]:
joined_df = pd.read_csv(PATH/'joined_sample.csv', index_col=None)

display(len(joined_df))
display(joined_df.head())
display(joined_df.describe().T)

10000

,business_id,cool,date,funny,review_id,stars,text,useful,user_id,user_average_stars,...,business_hours,business_is_open,business_latitude,business_longitude,business_name,business_neighborhood,business_postal_code,business_review_count,business_stars,business_state
0,8jpIK1WHmzzbXPaK51GenQ,1,2012-08-08,3,W7wcVRiw5T8TMrmGnxPsxQ,4,I've been here at least 10 times ... I like it...,1,g6gTSnUKZIxLZPQVrFKscw,4.14,...,"{'Tuesday': '6:30-14:30', 'Wednesday': '6:30-1...",0,33.320994,-111.912682,Dessie's Cafe,NaN,85226,67,3.5,AZ
1,wH4Q0y8C-lkq21yf4WWedw,0,2015-01-31,0,emypFL3PJjQBcllPZw_d5A,5,"Although I had heard of Nekter, mainly from se...",2,LAEJWZSvzsfWJ686VOaQig,5.00,...,"{'Monday': '6:30-20:0', 'Tuesday': '6:30-20:0'...",1,33.580474,-111.881062,Nekter Juice Bar,NaN,85260,59,4.0,AZ
2,cRMC2eQ9CP6ivhEY8EdaGg,1,2010-09-13,0,5X5ISEAp6HFTpMd_wlq_9w,3,Last week I met up with a highschool friend fo...,1,TwilnpgwW43r9-O2AS4PDQ,3.14,...,"{'Monday': '12:0-21:0', 'Tuesday': '12:0-21:0'...",0,43.664193,-79.380196,Chino Locos,Church-Wellesley Village,M4Y 2C5,34,3.5,ON
3,zunMkZ4U2eVojempQtLngg,1,2014-03-07,0,OGekU1U_wWgV--zL2gEgYw,4,A friend and I were driving by and decided to ...,1,eITkQlKYsYqOBASP-QS0iQ,3.72,...,"{'Monday': '11:0-1:0', 'Tuesday': '11:0-1:0', ...",0,33.639158,-112.185110,The Australian AZ,NaN,85308,26,2.5,AZ
4,1vLf-v7foAu3tJ7vAEoKdA,0,2014-11-26,1,tTe2cLFmpkLop3wKcT0Zgw,5,Our Bulldog LOVES this place and so do we! Won...,0,l3okl_UjyNdqRKAzYGdWaA,2.95,...,"{'Monday': '7:30-19:0', 'Tuesday': '7:30-19:0'...",1,33.582848,-111.929296,Lori's Grooming,NaN,85254,148,5.0,AZ


,count,mean,std,min,25%,50%,75%,max
cool,10000.0,0.558300,1.975650,0.000000,0.000000,0.000000,0.000000,70.000000
funny,10000.0,0.534500,4.304098,0.000000,0.000000,0.000000,0.000000,388.000000
stars,10000.0,3.721300,1.455211,1.000000,3.000000,4.000000,5.000000,5.000000
useful,10000.0,1.368800,3.678959,0.000000,0.000000,0.000000,2.000000,212.000000
user_average_stars,10000.0,3.739603,0.802876,1.000000,3.400000,3.810000,4.200000,5.000000
user_compliment_cool,10000.0,37.518800,308.275809,0.000000,0.000000,0.000000,2.000000,13014.000000
user_compliment_cute,10000.0,1.695100,31.125260,0.000000,0.000000,0.000000,0.000000,2250.000000
user_compliment_funny,10000.0,37.518800,308.275809,0.000000,0.000000,0.000000,2.000000,13014.000000
user_compliment_hot,10000.0,25.273900,240.013823,0.000000,0.000000,0.000000,1.000000,9259.000000
user_compliment_list,10000.0,1.222300,27.808089,0.000000,0.000000,0.000000,0.000000,2259.000000


## Use and test our MixedTabularList ItemList with the Data Block API

In [11]:
cat_cols = ['business_id', 'user_id', 'business_stars', 'business_postal_code', 'business_state']
cont_cols = ['useful', 'user_average_stars', 'user_review_count', 'business_review_count']
txt_cols = ['text']

dep_var = ['stars']

procs = [FillMissing, Categorify, Normalize]

**Step 1: Define the source of your inputs**

In [12]:
il = MixedTabularList.from_df(joined_df, cat_cols, cont_cols, txt_cols, vocab=None, procs=procs, path=PATH)

In [13]:
print(f'CATS:\n{il.cat_names}')
print(f'CONTS:\n{il.cont_names}')
print(f'TEXT COLS:\n{il.text_cols}')
print(f'PROCS:\n{il.procs}')
print('')
print(il.get(0))

CATS:
['business_id', 'user_id', 'business_stars', 'business_postal_code', 'business_state']
CONTS:
['useful', 'user_average_stars', 'user_review_count', 'business_review_count']
TEXT COLS:
['text']
PROCS:
[<class 'fastai.tabular.transform.FillMissing'>, <class 'fastai.tabular.transform.Categorify'>, <class 'fastai.tabular.transform.Normalize'>]

business_id                                         8jpIK1WHmzzbXPaK51GenQ
user_id                                             g6gTSnUKZIxLZPQVrFKscw
business_stars                                                         3.5
business_postal_code                                                 85226
business_state                                                          AZ
useful                                                                   1
user_average_stars                                                    4.14
user_review_count                                                       26
business_review_count                              

**Step 2: Split your dataset into training and validation `ItemList`s**

This is going to trigger the `ItemList.new()` method getting called for each `ItemList` it needs to create (e.g., train, validation).  Here it will be called 2x, once to create the training dataset and then to create the validation dataset.

In [14]:
ils = il.split_by_rand_pct(valid_pct=0.1, seed=42)

In [15]:
len(ils.train), len(ils.valid), ils.path

(9000, 1000, PosixPath('data/yelp_dataset'))

**Step 3: Add your labels (your targets or "y" values)**

This will grab the targets (the "y") for each `ItemList` in your `ItemLists` object (e.g, `.train`, `.valid`) and build a `LabelList(Dataset)` for each accordingly that is then combined in and returned in a `LabelLists` object.

You'll notice that the processor is created 1x but that .process is called 2x.  *Why?* So that the preprocessing defined by the training data is applied to the validation and optionally the test data later on.

In [16]:
ll = ils.label_from_df(dep_var)

In [17]:
type(ll), type(ll.train), len(ll.lists)

(fastai.data_block.LabelLists, fastai.data_block.LabelList, 2)

In [18]:
ll.train

LabelList (9000 items)
x: MixedTabularList
business_id wH4Q0y8C-lkq21yf4WWedw; user_id LAEJWZSvzsfWJ686VOaQig; business_stars 4.0; business_postal_code 85260; business_state AZ; useful 0.1650; user_average_stars 1.5663; user_review_count -0.3320; business_review_count -0.3931; Text: xxbos xxmaj although i had heard of xxmaj xxunk , mainly from seeing it xxunk in health conscious friends ' xxup ig posts , i had never tried it . xxmaj this location is rather new and conveniently located to me , so i gave it a try . xxmaj it 's xxup amazing ! xxmaj staff and the customer service they provide are phenomenal , and the having tried the juices ( fresh cold - pressed to order ) , smoothies & acai bowls , i 'm in love . xxmaj my 3 year old daughter even enjoyed the xxmaj pink xxmaj flamingo smoothie ( which they sell in kids size for little bellies ) . xxmaj they also sell pre - made detox and protein drinks in a small ready - to - go cooler area by the check out , although i have n't tried the

In [19]:
ll.train.x[0], ll.train.y[0], ll.train.x.codes[0], ll.train.x.cat_names, ll.train.x.text_ids[0]

(MixedTabularLine business_id wH4Q0y8C-lkq21yf4WWedw; user_id LAEJWZSvzsfWJ686VOaQig; business_stars 4.0; business_postal_code 85260; business_state AZ; useful 0.1650; user_average_stars 1.5663; user_review_count -0.3320; business_review_count -0.3931; Text: xxbos xxmaj although i had heard of xxmaj xxunk , mainly from seeing it xxunk in health conscious friends ' xxup ig posts , i had never tried it . xxmaj this location is rather new and conveniently located to me , so i gave it a try . xxmaj it 's xxup amazing ! xxmaj staff and the customer service they provide are phenomenal , and the having tried the juices ( fresh cold - pressed to order ) , smoothies & acai bowls , i 'm in love . xxmaj my 3 year old daughter even enjoyed the xxmaj pink xxmaj flamingo smoothie ( which they sell in kids size for little bellies ) . xxmaj they also sell pre - made detox and protein drinks in a small ready - to - go cooler area by the check out , although i have n't tried these yet . xxmaj and there 

In [20]:
len(ll.train.x.vocab.itos), len(ll.valid.x.vocab.itos)

(15256, 15256)

**Step 6: Build your DataBunch**

We're skilling steps 4 (add a test dataset) and 5 (apply data augmentation) since we have neither a test set or any transforms we need to apply to the data.

In [21]:
data_bunch = ll.databunch(bs=64)
b = data_bunch.one_batch()
len(b), len(b[0]), len(b[0][0]), len(b[0][1]), len(b[0][1]), b[1].shape

(2, 3, 64, 64, 64, torch.Size([64]))

`len(b) = 2`:  the inputs and the targets

`len(b[0]) = 3`: the three things in the input (cats, conts, text_ids)

`len(b[0][0|1|2|]) = 64`: there are 64 of each of the 3 things (so there is a list 64 categorical tensors followed by a list of 64 continuous tensors that is followed by a list of 64 text tensors)

The shape length of the categorical and continuous tensors are the same for every batch, whereas the shape of the numericalized token ids will be the same *per* batch thanks to the `mixed_tabular_pad_collate` function above.  This fulfills the requirement that each of the inputs be a squared matrix per batch.

In [22]:
b[0][0][0], b[0][1][0], b[0][2][0]

(tensor([4278, 7755,    3,  363,    9]),
 tensor([-0.3635, -2.1604, -0.3377, -0.4415]),
 tensor([    1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     2,     6,
          1807,     7,   346,    20,    13,    35,   897,    28,   188,    14,
           178,   195,    11,    29,    35,    51,  2089,   236,     9,     5,
            29,    92,    35,    14,   265,  1344,    24,    13,    35,   323,
            14,   395,   195,   342,   723,     7,   346,     9,   409,  1276,
             9,    13,    96,  2089,    12,    11,   341,    15,   888,    14,
           265,  1344,    47,   104,     9,    13,    96,    14,   155,   265,
            11,    13,   244,    15,   467,    17,  4734,    11,   239,     9,
            13,    96,    23,  1993,    11,

The above shows the categorical, continuous, and token ids for the first item in the batch

In [23]:
data_bunch.show_batch()

business_id,user_id,business_stars,business_postal_code,business_state,useful,user_average_stars,user_review_count,business_review_count,target
f11oGgqiYgcAYz1jYqFgSA,JdOQU7ryGozcWjvoGJSpHw,4.5,M5S 1G4,ON,-0.3635,0.3241,-0.3263,-0.2977,4
fN5uMqVJChcovlp4zIOL_g,1NRA_FMPNhCdsG-cFi-yaw,4.0,89131,NV,-0.3635,-0.9182,-0.3149,-0.3918,4
1Lbq92b6B9z1FYk5mS2R9Q,1RCo1B8YVPtzRDTrm3AyDw,3.5,89146,NV,-0.3635,0.3489,-0.0692,-0.3829,4
Rd11Bosr8JkfoUhZLzpuxg,7EiblY9N50yaE4Th6fHAIg,4.5,89109,NV,-0.3635,0.4855,-0.3263,0.4086,5
pomGBqfbxcqPv14c3XH-ZQ,HJj82f-csBI7jjgenwqhvw,4.0,89143,NV,1.4862,-0.1977,2.2958,0.9635,5


Because we included the `Normalize` proc, notice that the continuous variables are normalized *per dataset*: 
`(x - x.mean) / x.std`